In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

import evaluate
import torch
import numpy as np

In [ ]:
model_checkpoint = "NousResearch/Llama-2-7b-chat-hf"

id2label = {0:"Negative", 1:"Positive"}
label2id = {"Negative":0, "Positive":1}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

In [ ]:
dataset = load_dataset("shawhin/imdb-truncated")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [ ]:
def tokenize_function(examples):
  text = examples["text"]

  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
      text,
      return_tensors="np",
      truncation=True,
      max_length=512
  )

  return tokenized_inputs

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]' })
  model.resize_token_embeddings(len(tokenizer))

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)

  return {"accuracy": accuracy.compute(predictions=predictions,references=labels)}

In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                         r=4,
                         lora_alpha=32,
                         lora_dropout=0.01,
                         target_modules=['q_lin'])

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
le = 1e-3
batch_size = 4
num_epochs = 5

In [ ]:
training_args = TrainingArguments(
    output_dir=model_checkpoint + "-lora-text-classification",
    learning_rate=le,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
text_list = ["It was good.", "Not a fan, don't recomended.",
             "Better than the first one.", "This is not worth watching even once.",
             "This one is a pass."]

In [ ]:
print("Trained model predictions:")
print("--------------------------")

for text in text_list:
  inputs = tokenizer.encode(text, return_tensors="pt")

  logits = model(inputs).logits
  predictions = torch.max(logits,1).indices

  print(text + "-" + id2label[predictions.tolist()[0]])